In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

In [ ]:
bs = 64
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [ ]:
path  = Path('../input/jpeg-melanoma-512x512')
path

In [ ]:
df =pd.read_csv(path/'train.csv')
df

In [ ]:
df = df.drop(columns=['patient_id','sex','age_approx','anatom_site_general_challenge','diagnosis','benign_malignant','tfrecord','width','height'])


In [ ]:
#df = df.drop(columns=['patient_id'])

In [ ]:
train = path/'train'
train.ls()

In [ ]:
df.columns = ['name','label']

In [ ]:
df

In [ ]:
data = pd.DataFrame(df[df['label']==1])

In [ ]:
data

In [ ]:
data2 = pd.DataFrame(df[df['label']==0])
data2 = data2[:584]

In [ ]:
data2

In [ ]:
data = data.append(data2)
data

In [ ]:
df2 = data

In [ ]:
tfms = get_transforms(flip_vert=True)

In [ ]:
src = (ImageList.from_df(df2,path, suffix ='.jpg', folder = 'train')
    .split_by_rand_pct(0.2)
      .label_from_df()
      )

In [ ]:
data = (src.transform(tfms, size =256).databunch(bs=bs).normalize(imagenet_stats))

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[error_rate,accuracy]);learn.model

In [ ]:
learn.model_dir = "/kaggle/working"
learn.lr_find()


In [ ]:
learn.recorder.plot()

In [ ]:
lr=2e-4

In [ ]:
learn.fit_one_cycle(5,slice(lr))

In [ ]:
#learn.model_dir = "/kaggle/working" # Changing learn model_dir to /kaggle/working
learn.save('stage-1') #Saving model

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(1e-6,1e-5))

In [ ]:
learn.save('stage-2')

In [ ]:
melanoma_stats = ([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])

In [ ]:
data = (src.transform(tfms, size =512).databunch(bs=16).normalize(melanoma_stats))

In [ ]:
learn.data = data
data.train_ds[0][0].shape

In [ ]:
learn.freeze()


In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
lr = 2e-4

In [ ]:
learn.fit_one_cycle(5,slice(lr))

In [ ]:
learn.save('stage-3')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, slice(1e-6))

In [ ]:
learn.save('stage-4')

In [ ]:
learn.export('/kaggle/working/export.pkl')

In [ ]:
learner = load_learner('/kaggle/working')

In [ ]:
img = open_image(path/'test/ISIC_0052060.jpg')
pred_class,pred_idx,outputs = learner.predict(img)

# Get the probability of malignancy

prob_malignant = float(outputs[1])

print(pred_class)
print(prob_malignant)

In [ ]:
test = os.listdir(path/'test')
test.sort(key=lambda f: int(re.sub('\D', '', f)))

with open('/kaggle/working/submission.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_name', 'target'])
    
    for image_file in test:
        image = os.path.join(path/'test', image_file) 
        image_name = Path(image).stem

        img = open_image(image)
        pred_class,pred_idx,outputs = learner.predict(img)
        target = float(outputs[1])
        
        writer.writerow([image_name, target])